# Gensim

환경: Python2.7

#### Topic modeling : 문서의 주제를 찾는 모델링 방법

- **LSI(Latent Simantic Indexing)**
- **LDA(Latent Dirichlet Allocation)**
- TF-IDF
- HDP(Hierarchical Dirichlet Process)
- Word Representation (Word2Vec)
- Document Representation (Doc2Vec)

Why?
 - 숨겨진 토픽의 패턴을 찾기 위해 토픽 모델을 이용

Dependency
 - Theano
 - scipy
 - numpy

Python 3.4, 2.7에서 사용 가능

Python 3.5의 경우는 Theano가 Python 3.4버젼까지밖에 Dependency가 제공되지 않으므로 안깔림

<img src="images/gensim_install.jpg">

<img src="images/gensim_install2.jpg">

## -- Install Finished

## 1. Corpus(말뭉치) 생성

- Corpus: 연구를 위한 자료나 정보의 모음

NLP(Natural Language Processing)에서 사용하는 Corpus는 문서 전체에 대한 단어들의 집합

### 가) Preprocessing

In [181]:
from gensim import corpora, models, similarities
import pprint
documents = ["Human machine interface for lab abc computer applications",
              "A survey of user opinion of computer system response time",
              "The EPS user interface management system",
              "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

-- 전치사 제거

In [182]:
stoplist = set('for a of the and to in'.split())

In [183]:
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

In [184]:
texts

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
 ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'management', 'system'],
 ['system', 'human', 'system', 'engineering', 'testing', 'eps'],
 ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'],
 ['generation', 'random', 'binary', 'unordered', 'trees', 'trees'],
 ['intersection', 'graph', 'paths', 'trees'],
 ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'],
 ['graph', 'minors', 'survey']]

단어 카운트 숫자가 적은 경우 제거

In [185]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [186]:
texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees', 'trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

### 나) 사전 생성

In [187]:
from gensim import corpora

corpora는 말뭉치(corpus) Set을 만들기 위해 사용

corpora.Dictionary는 Key, Value값 형태로 Key에는 단어가 들어가며 Value에는 단어의 Frequency에 따라 sorting 후 a-z형태로 나열하여 순서를 나타냄

In [188]:
dict1 = corpora.Dictionary(texts)
dict1.save('./deerwester.dict') 
pprint.pprint(dict1.token2id)

{u'computer': 1,
 u'eps': 8,
 u'graph': 10,
 u'human': 2,
 u'interface': 0,
 u'minors': 11,
 u'response': 3,
 u'survey': 5,
 u'system': 6,
 u'time': 4,
 u'trees': 9,
 u'user': 7}


## 2. Bag of words

<img src="images/BOW.jpg">

BOW(Bag of words)를 통해 문장에서의 단어 분포도를 찾을 수 있다

In [189]:
pprint.pprint(dict1.token2id)

{u'computer': 1,
 u'eps': 8,
 u'graph': 10,
 u'human': 2,
 u'interface': 0,
 u'minors': 11,
 u'response': 3,
 u'survey': 5,
 u'system': 6,
 u'time': 4,
 u'trees': 9,
 u'user': 7}


In [190]:
# text문장을 추출
[dict1.doc2bow(text) for text in texts]

# BOW를 통한 단어 파악
print(dict1.doc2bow(text))
print(text)

[(5, 1), (10, 1), (11, 1)]
['graph', 'minors', 'survey']


In [191]:
corpus = [dict1.doc2bow(text) for text in texts]
pprint.pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 2)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]


## 3. Corpus in Matrix Maret Format

In [192]:
# 일반적인 Market Matrix형태
corpora.MmCorpus.serialize('./deerwester.mm',corpus)

# SVM형태의 corpus를 나타낼 때 사용하는 형태
corpora.SvmLightCorpus.serialize('./deerwester.svmlight',corpus)

# EM 알고리즘(expectation-maximization algorithm, 기대값 최대화)을 나타낼 때 사용 : 
corpora.BleiCorpus.serialize('./deerwester.lda-c',corpus)

# Gibbs 샘플링을 통한 LDA분석에 사용
corpora.LowCorpus.serialize('./deerwester.low',corpus)

일반적인 Corpus사용

In [193]:
corpora.MmCorpus.serialize('./deerwester.mm',corpus)
mm=corpora.MmCorpus('./deerwester.mm')

In [194]:
print(mm)

MmCorpus(9 documents, 12 features, 28 non-zero entries)


In [196]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            yield dictionary.doc2bow(line.lower().split())

In [197]:
corpus_memory_friendly = MyCorpus()

In [199]:
dictionary = corpora.Dictionary(line.lower().split() for line in open('data/mycorpus.txt'))
pprint.pprint(dictionary.token2id)

{u'a': 8,
 u'abc': 0,
 u'and': 19,
 u'applications': 4,
 u'binary': 28,
 u'computer': 5,
 u'engineering': 20,
 u'eps': 18,
 u'error': 26,
 u'for': 1,
 u'generation': 27,
 u'graph': 35,
 u'human': 6,
 u'in': 34,
 u'interface': 7,
 u'intersection': 33,
 u'iv': 40,
 u'lab': 2,
 u'machine': 3,
 u'management': 17,
 u'measurement': 24,
 u'minors': 36,
 u'of': 9,
 u'opinion': 14,
 u'ordering': 37,
 u'paths': 32,
 u'perceived': 23,
 u'quasi': 41,
 u'random': 29,
 u'relation': 25,
 u'response': 15,
 u'survey': 11,
 u'system': 10,
 u'testing': 21,
 u'the': 16,
 u'time': 13,
 u'to': 22,
 u'trees': 31,
 u'unordered': 30,
 u'user': 12,
 u'well': 39,
 u'widths': 38}


## 4. LSI(Latent Simantic Indexing)

LSI는 문서에 포함되어 있는 키워드를 기록할 뿐만 아니라, 문서 컬렉션 전체를 평가하여 어떤 문서가 비슷한 단어를 포함하고 있는 지를 찾아낸다. LSI는 많은 단어를 공유하는 문서들이 결과적으로 의미면에서 (semantically) 가까운 것으로 간주하며, 공유하는 단어가 적으면 의미적으로 먼 것으로 여김 

LSI 알고리즘은 단어의 뜻을 이해하지는 못하지만, 단어들이 보여주는 패턴을 인식 함

- LSI는 문서에 포함되어 있는 키워드를 기록할 뿐만 아니라, 문서 컬렉션 전체를 평가하여 어떤 문서가 비슷한 단어를 포함하고 있는지를 찾아냄

#### Example)

T1)The Neatest Little **Guide** to **Stock Market Investing**

T2) **Investing** For **Dummies**, 4th Edition

T3) The Little **Book** of Common Sense **Investing**: The Only Way to Guarantee Your Fair Share of **Stock Market** Returns

T4) The Little **Book** of **Value Investing**

T5) **Value Investing**: From Graham to Buffett and Beyond

T6) **Rich Dad’s Guide** to **Investing**: What the **Rich** Invest in, That the Poor and the Middle Class Do Not!

T7) **Investing** in **Real Estate**, 5th Edition

T8) **Stock Investing** For **Dummies**

T9) **Rich Dad’s** Advisors: The ABC’s of **Real Estate Investing**: The Secrets of Finding Hidden Profits Most Investors Miss

#### 진행순서

가. 위에서 나타낸 1~9까지의 document 중 단어가 2개 이상인 것만 간추림

나. 아래의 그림처럼 Vector를 나타낼 수 있음



<img src="images/matrix1.jpg">

<img src="images/xygraph2.png" width="500" height="500">

In [200]:
lsi = gensim.models.lsimodel.LsiModel(corpus=mm, id2word=dictionary, num_topics=400)

In [201]:
lsi.print_topics(5)

[(0,
  u'0.644*"human" + 0.403*"interface" + 0.301*"a" + 0.265*"machine" + 0.265*"applications" + 0.240*"for" + 0.221*"lab" + 0.206*"computer" + 0.198*"abc" + 0.039*"system"'),
 (1,
  u'-0.779*"of" + -0.501*"system" + -0.335*"survey" + -0.136*"computer" + 0.074*"human" + 0.053*"a" + 0.040*"lab" + 0.031*"abc" + -0.012*"machine" + -0.012*"applications"'),
 (2,
  u'-0.428*"machine" + -0.428*"applications" + 0.390*"human" + 0.354*"a" + -0.328*"interface" + 0.305*"lab" + -0.276*"computer" + -0.168*"for" + 0.162*"of" + 0.151*"abc"'),
 (3,
  u'0.567*"of" + -0.494*"survey" + -0.464*"system" + -0.374*"computer" + 0.180*"interface" + 0.138*"machine" + 0.138*"applications" + -0.061*"lab" + -0.057*"human" + 0.035*"abc"'),
 (4,
  u'0.593*"for" + 0.559*"abc" + 0.412*"lab" + -0.338*"human" + -0.186*"a" + -0.087*"interface" + -0.068*"applications" + -0.068*"machine" + -0.015*"system" + -0.014*"survey"')]

### LSI 실제 구하는 방법

<img src="images/Example1.jpg">

<img src="images/Example2.jpg">

<img src="images/Example3.jpg">

<img src="images/Example4.jpg">

<img src="images/Example5.jpg">

## 5. LDA(Latent Dirichlet Allocation)

**사전확률, 사후확률, 기대값, 베이즈 룰 등은 생략**

-- 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지에 대한 확률 모형

-- 미리 알고 있는 주제별 단어수 분포를 바탕으로, 주어진 문서에서 발견된 단어수 분포를 분석함으로서 해당 문서가 어떤 주제들을 함께 다루고 있을지를 예측할 수 있다

- Latent : 잠재적인
- Dirichlet : 디리클레  
- Allocation : 할당

#### 나에게 1만개의 문서가 있다 가정했을 때 1만개의 문서를 구성하고 있는 어떠한 토픽들을 알아내고 싶다고 했을 때 사용할 수 있는 모델

 **"주말 더 덥다, 일요일엔 장맛비"** 라는 기사를 보았을 때
 
- 일요일, 기온, 날씨, 주말 이라는 연관된 토픽들을 다룸
- 이것처럼 모든 문서는 하나의 토픽에만 속하는 것이 아닌 다른 토픽들의 혼합모델(Mixture model)로 정의할 수 있음

LDA는 어떤 문서에 대해 주제 벡터(Theata)가 있고, 앞에서부터 단어를 하나씩 채울 때마다 주제벡터로부터 하나의 주제를 선택하고, 다시 그 주제로부터 단어를 선택하는 방식으로 문서 생성과정을 모델링 

<img src="images/plate.png">

$$ alpha,beta : 코퍼스(말뭉치) 단위로 정해지는 값 $$
$$ alpha는 기존문서이며, beta는 토픽을 형성하는 단어의 집합 $$

$$ \theta : 주제 벡터 $$ 

$$ w : 단어 $$

1. 문서의 **토픽들**을 정하고

2. **각 단어**의 **토픽**을 정하며

3. 토픽을 형성하는 단어의 집합에서 단어를 **뽑아** 문서에 쓴다

<img src="images/result.png">

In [78]:
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=dictionary, num_topics=10, update_every=1, chunksize=10, passes=1)

In [79]:
lda.print_topics(4)

[(1,
  u'0.108*system + 0.108*for + 0.108*applications + 0.108*abc + 0.108*lab + 0.108*survey + 0.010*of + 0.010*computer + 0.010*interface + 0.010*machine'),
 (8,
  u'0.024*of + 0.024*system + 0.024*computer + 0.024*machine + 0.024*survey + 0.024*applications + 0.024*for + 0.024*interface + 0.024*abc + 0.024*lab'),
 (5,
  u'0.108*applications + 0.108*computer + 0.108*interface + 0.108*abc + 0.108*machine + 0.108*human + 0.010*of + 0.010*system + 0.010*lab + 0.010*survey'),
 (6,
  u'0.177*system + 0.177*of + 0.016*computer + 0.016*interface + 0.016*survey + 0.016*lab + 0.016*machine + 0.016*applications + 0.016*for + 0.016*abc')]